# Lab 6: Random Forest Regression on Malagasy Mammal Traits

### Introduction
 In Lab 5, we used functional trait data on bird species to predict their presence in a given climate type. This week we are using regression tree models on a similar dataset, this one on mammal species.  Our goal is to predict the total number of climate types each mammal species inhabits.

You will compare performance of a single decision tree with a tuned random forest to get a sense of the effectiveness of ensemble methods with optimized hyperparameter values.

There is a fair amount of missing data in this set, which can lead to low reliability of feature splits, increased overfitting, and decreased accuracy of tree models. You will address this issue with a combination of variable omission and imputation.


In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from matplotlib.colors import ListedColormap, Normalize
import matplotlib.cm as cm

### Step 1: Read data and retain a subset of the columns
Similar to last week, we will need to take some steps to get the data in a usable form. We again reformat the diet-related variable names to match this set: `Diet_Vertebrates`,
    `Diet_Fruits`, `Diet_Flowers`, `Diet_Seeds`, `Diet_Plants`, `Diet_Other`. Then drop columns that are not relevant functional traits or climate type variables.   Next, drop any remaining  variables that have greater than 40% of the observations missing. Print the final dataframe shape. 

In [10]:
dat = pd.read_csv("/courses/EDS232/Data/MamTraitData.csv", encoding="ISO-8859-1").rename(columns={
    'Diet: invertebrates': 'Diet_Invertebrates',
    'Diet: vertebrates': 'Diet_Vertebrates',
    'Diet: fruits': 'Diet_Fruits',
    'Diet: flower/nectar/pollen/gums': 'Diet_Flowers',
    'Diet: seeds': 'Diet_Seeds',
    'Diet: other plant materials': 'Diet_Plants',
    'Diet: scavenge; garbage; carrion; carcasses': 'Diet_Other'
})

# Drop variables that have greater that 40% observations missing
dat = dat.dropna(thresh = len(dat)*0.6, axis = 1)

# Print the final df shape
print(f"Final df shape: {dat.shape}")

# Show the head of the df
dat.head(3)

Final df shape: (214, 17)


,AdultBodyMass,Diet_Invertebrates,Diet_Vertebrates,Diet_Fruits,Diet_Flowers,Diet_Seeds,Diet_Plants,Diet_Other,HabitatBreadth,ActivityCycle,ForagingStratum,LitterSize,Dry,Humid,Montane,Subarid,Subhumid
0,78.0,yes,no,yes,yes,no,yes,no,1.0,2.0,3.0,1.0,0,1,1,0,1
1,1082.0,no,no,yes,no,no,yes,no,1.0,1.0,3.0,NaN,0,1,0,0,1
2,946.0,no,no,yes,no,no,yes,no,2.0,1.0,3.0,NaN,1,0,0,0,0


### Step 2: Imputation with KNN
Now impute the values of the missing data in the remaining numeric columns using k-nearest neighbors with `KNNImputer()`. Use the 5 nearest neighbors. This allows us to implement the knn algorithm to predict the missing values for an observation based on similar complete observations.Perform and print a check to ensure that there are no more NA values. 

In [13]:
# Separate numeric and categorical columns
numeric_cols = dat.select_dtypes(include=['number']).columns
categorical_cols = dat.select_dtypes(include=['object']).columns

# Create imputers for numeric and categorical columns
numeric_imputer = KNNImputer(n_neighbors = 5)
categorical_imputer = SimpleImputer(strategy='most_frequent') 

# Impute missing values
dat[numeric_cols] = pd.DataFrame(numeric_imputer.fit_transform(dat[numeric_cols]))
dat[categorical_cols] = pd.DataFrame(categorical_imputer.fit_transform(dat[categorical_cols]))

# Look at NAs after imputing
print(f"NAs after imputing:\n{dat.isna().sum()}")

NAs after imputing:
AdultBodyMass         0
Diet_Invertebrates    0
Diet_Vertebrates      0
Diet_Fruits           0
Diet_Flowers          0
Diet_Seeds            0
Diet_Plants           0
Diet_Other            0
HabitatBreadth        0
ActivityCycle         0
ForagingStratum       0
LitterSize            0
Dry                   0
Humid                 0
Montane               0
Subarid               0
Subhumid              0
dtype: int64


### Step 3: Create target variable and numerical encoding 
Create the target variable (`nClimates`) representing the  number of climates in which a species was present. Then finish preparing the data to be used in our models.

- Create a new column `nClimates` that combines the information of the five climate type variables
- Drop the original `Dry`, `Humid`, `Montane`, `Subarid`, and `Subhumid` columns
- Encode categorical variables
- Split the dataset into training and test sets. Use a random state of 808. 

In [14]:
# Create new col `nClimates` that combines the info of the 5 climate type variables
dat['nClimates'] = dat['Dry'] + dat['Humid'] + dat['Montane'] + dat['Subarid'] + dat['Subhumid']

# Drop the original cols
df = dat.drop(['Dry', 'Humid', 'Montane', 'Subarid', 'Subhumid'], axis = 1)

# Encode categorical variables - with a 4 loop!
for col in categorical_cols:
    label_encoder = LabelEncoder()
    df[col] = label_encoder.fit_transform(df[col])

# Define features
X = df.drop('nClimates', axis = 1)

# Define targets
y = df['nClimates']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 808)

### Step 4: Train a decision tree
Train a single decision tree to predict the number of climate types inhabited by each species.

- Train a `DecisionTreeRegressor`.  Let's apply a constraint on its growth by not allowing the tree to grow further than 5 levels. Use a random state of 808. 
- Make predictions
- Visualize the decision tree using `plot_tree`
- Evaluate and print its performance using mse

In [25]:
# Train a single decision tree
dt = DecisionTreeRegressor(max_depth = 5, random_state = 808)
dt.fit(X_train, y_train)

# Make predictions
y_pred = dt.predict(X_test)


# Plot a decision tree
plt.figure(figsize=(12, 15), dpi=700)
plot_tree(dt, feature_names=X.columns, filled=True, fontsize=6,
          rounded=True, precision=2)
plt.title('Decision Tree Predicting the number of Climate Types Inhabited by Species')
plt.show()

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f"DT Regressor MSE: {mse:.3f}")

DT Regressor MSE: 1.832


Based on the way this tree looks, what would you guess the most important variable to be?

*Your answer here.*

### Step 5: Random forest with hyperparameter tuning
Now we'll use a more sophisticated ensemble approach where we optimize the values of key hyperparameters that govern tree growth.

- Define a parameter grid that includes the follow parameters and values:
  -  (sqrt(p), 6, and no set #) of the number of features to try at each split
  -  (50, 100, 200) total trees (learners)
  -  a maximum tree depth of (3,4,5,6,7)
  -  (2,5,10) minimum samples per split
  -  (1,2,4) minimum samples per leaf
  
- Use `GridSearchCV` with 5-fold cross-validation and mse (`neg_mean_squared_error`) to find the best combination of parameter values
- Train the best estimator.

In [ ]:
...

### Step 6: Make predictions and evaluate random forest model
Use the optimized random forest model (`best_rf`) to predict the number of climates a species inhabits and analyze its performance.

- Use `best_rf` and `predict()` method on `X_test` to generate predictions
- Print the best set of parameter values using thte `best_params` method
- Compute and print the mse to evaluate model performance
- Extract feature importance from the trained random forest model
- Visualize feature importance using a bar plot

In [ ]:
...

Which model had better accuracy, the single tree or the ensemble.  Interpret which traits have the most influence on species' climate adaptability.

*Your answer here.*